In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir("..")

In [3]:
import torch
from omegaconf import OmegaConf
from train_utils import prepare_data, prepare_model, prepare_optimizer, train, set_seeds

In [4]:
# set seeds
set_seeds()

# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [5]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        image_folder="datasets/train/images/",
        gt_folder="datasets/train/groundtruth/",
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        random_resized_crop_scale=(0.5, 1.0),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=False,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=False,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=False,
        degrees=10,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=False,  # TODO: Should it always be True?
        # Data loaders
        batch_size=2,
        train_size=0.8,
        val_size=0.1,
        test_size=0.1,
        # Model
        # ResNet101
        model_name="ResNet101",
        model_pretrained=False,
        model_save_name="models/checkpoints/deeplabv3_resnet101.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=0.1,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="deeplabv3-resnet101",
        entity="feeit",
    )
)

In [6]:
# prepare train, validation and test loaders
train_loader, val_loader, test_loader = prepare_data(args)
# prepare model
model = prepare_model(args)
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()
# prepare optimizer
optimizer = prepare_optimizer(model, args)

Using ToTensor.
Initializing ResNet101 model.


/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Initializing Adam optimizer with lr=0.1.


In [7]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet101(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (

In [8]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stefan-krsteski (feeit). Use `wandb login --relogin` to force relogin


Best model saved at step:  0
Best model saved at step:  100
Best model saved at step:  400
Best model saved at step:  500
Best model saved at step:  700
Best model saved at step:  900
Best model saved at step:  1400
Best model saved at step:  1600
Best model saved at step:  1800


Submission

In [9]:
# import transforms
from torchvision import transforms
from datasets.TestDataset import TestDataset
from models.DeepLabV3 import ResNet101
import matplotlib.pyplot as plt
import torch
from examples.mask_to_submission import *

In [10]:
MODEL = "models/checkpoints/deeplabv3_resnet101.pt"
model = ResNet101()

In [11]:
# Load the checkpoint
checkpoint = torch.load(MODEL)

model.load_state_dict(checkpoint)

model.eval()

ResNet101(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (

In [12]:
# path to the test folder
test_folder = "datasets/test/"

# define transformations
transform = transforms.Compose([transforms.ToTensor()])

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

In [13]:
# TODO FIX THIS, bad predictions

In [14]:
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [15]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)